# Определение уровня лояльности клиентов.

Задача:

Определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Подготовить дашборд с итогами опроса клиентов. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

Описание данных:

Таблица user

Содержит основную информацию о клиентах.

- user_id - Идентификатор клиента, первичный ключ таблицы
- lt_day - Количество дней «жизни» клиента
- age - Возраст клиента в годах
- gender_segment - Пол клиента (1 – женщина, 0 – мужчина)
- os_name - Тип операционной системы
- cpe_type_name - Тип устройства
- location_id - Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
- age_gr_id - Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
- tr_gr_id - Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
- lt_gr_id - Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
- nps_score - Оценка клиента в NPS-опросе (от 1 до 10)

Таблица location

Справочник территорий, в которых телеком-компания оказывает услуги.

- location_id - Идентификатор записи, первичный ключ
- country - Страна
- city - Город

Таблица age_segment

Данные о возрастных сегментах клиентов.

- age_gr_id - Идентификатор сегмента, первичный ключ
- bucket_min - Минимальная граница сегмента
- bucket_max - Максимальная граница сегмента
- title - Название сегмента

Таблица traffic_segment

Данные о выделяемых сегментах по объёму потребляемого трафика.

- tr_gr_id - Идентификатор сегмента, первичный ключ
- bucket_min - Минимальная граница сегмента
- bucket_max - Максимальная граница сегмента
- title - Название сегмента

Таблица lifetime_segment

Данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.

- lt_gr_id - Идентификатор сегмента, первичный ключ
- bucket_min - Минимальная граница сегмента
- bucket_max - Максимальная граница сегмента
- title - Название сегмента

# План проекта

1. Подключение к базе.

Необходимо получить доступ к базе данных. Данные выгрузили в SQLite — СУБД, в которой база данных представлена файлом. Для подключения к такой базе достаточно иметь доступ к файлу с расширением .db.

2. Выгрузка данных.

Подготовить данные для дашборда. Нужно собрать в одну витрину данные из разных таблиц. Эту витрину будем использовать для дашборда.

2.1 Написать запрос, который выгрузит следующие поля:

- user_id	Идентификатор клиента
- lt_day	Количество дней «жизни» клиента
- is_new	Поле хранит информацию о том, является ли клиент новым
- age	Возраст
- gender_segment	Пол (для удобства работы с полем преобразуем значения в текстовый вид)
- os_name	Тип операционной системы
- cpe_type_name	Тип устройства
- country	Страна проживания
- city	Город проживания
- age_segment	Возрастной сегмент
- traffic_segment	Сегмент по объёму потребляемого трафика
- lifetime_segment	Сегмент по количеству дней «жизни»
- nps_score	Оценка клиента в NPS-опросе
- nps_group	Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

2.2 Получившуюся таблицу сохранить как CSV-файл.

3. Создание дашборда в Tableau.

Построить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков. 

4. Ответить на вопросы с помощью дашборда. 

- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников?

## Подключение к базе.

In [1]:
# Импортируем нужные библиотеки
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# путь к БД на компьютере 
path_to_db_local = 'telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

# если путь на компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# если итоговый путь не пустой
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Выгрузка данных.
### Написание запроса.

In [3]:
# Сохраняем в переменную SQL-запрос
query = """
SELECT u.user_id AS user_id, 
        u.lt_day AS lt_day, 
        u.age AS age, 
        CAST(u.gender_segment AS varchar) AS gender_segment, 
        u.os_name AS os_name, 
        u.cpe_type_name AS cpe_type_name, 
        u.nps_score AS nps_score, 
        ass.title AS age_segment, 
        ts.title AS traffic_segment, 
        ls.title AS lifetime_segment, 
        l.country AS country, 
        l.city AS city, 
        CASE
            WHEN u.lt_day < 365 THEN 'new'
            ELSE 'old'
        END AS is_new, 
        CASE
            WHEN u.nps_score BETWEEN 9 AND 10 THEN 'сторонник'
            WHEN u.nps_score BETWEEN 7 AND 8 THEN 'нейтрал'
            ELSE 'критик'
        END AS nps_group 
    FROM user AS u
    LEFT JOIN location AS l ON l.location_id=u.location_id
    LEFT JOIN age_segment AS ass ON ass.age_gr_id=u.age_gr_id
    LEFT JOIN traffic_segment AS ts ON ts.tr_gr_id=u.tr_gr_id
    LEFT JOIN lifetime_segment AS ls ON ls.lt_gr_id=u.lt_gr_id
    WHERE l.country = 'Россия'; 
"""

In [4]:
# создаём датафрейм по данным запроса и выводим первые 3 строки таблицы
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,nps_score,age_segment,traffic_segment,lifetime_segment,country,city,is_new,nps_group
0,A0CMN5,2729,27.0,0.0,ANDROID,SMARTPHONE,6,03 25-34,16 60-65,08 36+,Россия,Архангельск,old,критик
1,A0CYVK,478,48.0,1.0,ANDROID,SMARTPHONE,7,05 45-54,06 10-15,06 13-24,Россия,Архангельск,old,нейтрал
2,A0HNQU,413,42.0,1.0,ANDROID,SMARTPHONE,9,04 35-44,08 20-25,06 13-24,Россия,Архангельск,old,сторонник


### Сохранение файла в формат csv.

In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Дашборд с результатами.

Ссылка:
https://public.tableau.com/views/_16928829199200/sheet18?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link